# Утилиты

In [46]:
import math
from typing import Callable

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.linalg import sqrtm

In [47]:
def calculate_condition_number(matrix: np.ndarray) -> float:
    """Вычисляем число обусловленности матрицы."""
    return linalg.cond(matrix)

In [48]:
def get_square_root_of_matrix(matrix: np.ndarray) -> np.ndarray:
    """Находим квадратный корень из матрицы."""
    return sqrtm(matrix)

In [49]:
def generate_matrix(
        element_factory: Callable[[int, int], float],
        power_factory: Callable[[int, int], float],
        size: int,
) -> np.ndarray:
    """
    Генерируем матрицу. 
    
    Элементы матрицы создаются с помощью функции element_factory, 
    которая принимает на вход номер строки и столбца, и возвращает элемент матрицы.

    Все элементы матрицы возводятся в соответсвующие степени, которые получаются из функции power_factory,
    получающей на вход номер строки и столбца.
    """
    return np.array(
        [
            [element_factory(row, column) ** power_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )

In [50]:
def get_matrix(size: int) -> np.ndarray:
    """Создаём обобщённую матрицу Вандермонда: a_k = k, b_k = k / 3."""
    return generate_matrix(
        element_factory=lambda row, column: row,
        power_factory=lambda row, column: column / 3,
        size=size
    )

In [51]:
def find_right_part(matrix: np.ndarray, solution: np.ndarray) -> np.ndarray:
    """Находим u: matrix * solution = u"""
    return matrix.dot(solution)

In [52]:
def find_solutions_using_regularization(matrix: np.ndarray, right_part: np.ndarray, alpha: float) -> pd.Series:
    """Находит решение СЛУ с помощью методов регуляризации."""
    solutions = []

    # First method
    transpose_matrix = np.transpose(matrix)

    new_matrix = transpose_matrix.dot(matrix) + alpha * np.eye(matrix.shape[0])
    new_right_part = transpose_matrix.dot(right_part)

    solutions.append(linalg.solve(new_matrix, new_right_part))

    # Second method
    square_root_matrix = get_square_root_of_matrix(matrix)
    transposed_square_root_matrix = np.transpose(square_root_matrix)
    inversed_square_root_matrix = linalg.inv(square_root_matrix)

    new_matrix = transposed_square_root_matrix.dot(square_root_matrix) + alpha * np.eye(square_root_matrix.shape[0])
    new_right_part = linalg.multi_dot([transposed_square_root_matrix, inversed_square_root_matrix, right_part])

    solutions.append(linalg.solve(new_matrix, new_right_part))

    return pd.Series(solutions)

In [53]:
def estimate_error(approximate_solution: np.ndarray, true_solution: np.ndarray) -> float:
    """Находим то, насколько мы ошиблись с реальным решением."""
    return linalg.norm(approximate_solution - true_solution)

# Поиск n

In [54]:
def banchmark_matrix(matrix: np.ndarray) -> pd.Series:
    """
    "Тестируем" матрицу.

    Находим число обусловленности для переданной матрицы matrix и для sqrt(matrix).
    Также вычисляем то, насколько мы ошиблись при вычислении квадратного корня.
    """
    square_root_matrix = get_square_root_of_matrix(matrix)

    cond_matrix = calculate_condition_number(matrix)
    cond_square_root_matrix = calculate_condition_number(square_root_matrix)
    error = linalg.norm(matrix - square_root_matrix.dot(square_root_matrix))

    return pd.Series([cond_matrix, cond_square_root_matrix, error])

In [55]:
matrix_data = pd.DataFrame(range(1, 20), columns=['n'])
matrix_data[['cond_A', 'cond_B', 'error']] = matrix_data.apply(
    func=lambda row: banchmark_matrix(get_matrix(row.n)),
    axis=1,
)
matrix_data

# matrix_data.to_csv('matrix_data.csv', index=False)

,n,cond_A,cond_B,error
0,1,1.000000e+00,1.000000e+00,0.000000e+00
1,2,1.859543e+01,4.321623e+00,7.691851e-16
2,3,3.234771e+02,1.801020e+01,2.349899e-15
3,4,6.092365e+03,7.862352e+01,2.239268e-14
4,5,1.265762e+05,3.634385e+02,2.761310e-14
5,6,2.869744e+06,1.767862e+03,9.481318e-14
6,7,6.983141e+07,8.965572e+03,9.935212e-14
7,8,1.798721e+09,4.703482e+04,5.290142e-13
8,9,4.855542e+10,2.537841e+05,2.688529e-12
9,10,1.363842e+12,1.402470e+06,6.242251e-12


In [56]:
fig = px.line(
    matrix_data[1:],
    x='n',
    y='error',
    log_y=True,
    title='Зависимость ошибки вычисления квадратного корня матрицы от её размера',
)
fig.update_xaxes(title='Размер матрицы')
fig.update_yaxes(tickformat='.e', title='Ошибка')
fig.add_hline(y=10 ** -11, line_color='red')
fig.update_layout(template=None)
fig

# fig.write_image('n-sqrt-error.svg')

Unsupported

# Проверка матриц

In [57]:
def check_eigenvalues(eigenvalues: np.ndarray) -> bool:
    # Проверяем, что собственные числа не равны 0
    if any(list(map(lambda number: math.isclose(number, 0, rel_tol=10 ** -12), eigenvalues))):
        return False

    # Проверяем, что собственные числа различны
    for first, second in zip(eigenvalues, eigenvalues[1:]):
        if math.isclose(first, second, rel_tol=10 ** -12):
            return False

    return True

In [58]:
def check_eigenvectors(eigenvectors: np.ndarray) -> bool:
    # Проверяем, что первый вектор не равен 0
    if any(list(map(lambda number: math.isclose(number, 0, rel_tol=10 ** -12), eigenvectors.T[0]))):
        return False

    # Проверяем, что соблюдается знакочередование
    for i, vector in enumerate(eigenvectors.T):
        is_sign_change = np.diff(np.sign(vector)) != 0
        if is_sign_change.astype(int).sum() != i:
            return False

    return True

In [59]:
def check_matrix(matrix: np.ndarray) -> bool:
    """Проверяем, что переданная матрица является осцилляционной."""
    eigenvalues, eigenvectors = linalg.eig(matrix)

    sort_order = np.flip(eigenvalues.argsort())
    eigenvalues = eigenvalues[sort_order]
    eigenvectors = eigenvectors[:, sort_order]

    return check_eigenvalues(eigenvalues) and check_eigenvectors(eigenvectors)

In [60]:
test_data = pd.DataFrame(range(1, 11), columns=['n'])
test_data['is_correct'] = test_data.apply(
    func=lambda row: check_matrix(get_matrix(row.n)),
    axis=1,
)
test_data

,n,is_correct
0,1,True
1,2,True
2,3,True
3,4,True
4,5,True
5,6,True
6,7,True
7,8,True
8,9,True
9,10,True


# Поиск alpha

In [61]:
def get_regularization_data(size: int) -> pd.DataFrame:
    matrix = get_matrix(size)
    right_part = find_right_part(matrix, np.ones(size))

    regularization_data = pd.DataFrame([10 ** -i for i in range(2, 12)], columns=['alpha'])
    regularization_data[['solution_1', 'solution_2']] = regularization_data.apply(
        func=lambda row: find_solutions_using_regularization(matrix, right_part, row.alpha),
        axis=1,
    )

    regularization_data['error_1'] = regularization_data.apply(
        func=lambda row: estimate_error(row.solution_1, np.ones(size)),
        axis=1
    )

    regularization_data['error_2'] = regularization_data.apply(
        func=lambda row: estimate_error(row.solution_2, np.ones(size)),
        axis=1
    )

    return regularization_data

In [62]:
regularization_data = get_regularization_data(10)
regularization_data

# regularization_data[['alpha', 'error_1', 'error_2']].to_csv('regularization_data.csv', index=False)

,alpha,solution_1,solution_2,error_1,error_2
0,1.000000e-02,"[0.9808723977608305, 0.9954039291695705, 1.005...","[0.9919296284982112, 0.9970874717584739, 0.999...",0.025725,0.008663
1,1.000000e-03,"[0.9960864130801098, 1.0004120705199042, 1.002...","[0.999108882153855, 0.9997876019182771, 1.0000...",0.005276,0.000920
2,1.000000e-04,"[0.9991863744926299, 1.0002855457871966, 1.000...","[0.9999059841087095, 0.9999876398891457, 1.000...",0.001349,0.000095
3,1.000000e-05,"[0.9998833031213801, 1.0000684481420354, 1.000...","[0.9999877210317494, 1.0000068217557827, 0.999...",0.000181,0.000017
4,1.000000e-06,"[0.9999891515138227, 1.0000156795611252, 0.999...","[0.9999943923050466, 1.000014923158066, 0.9999...",0.000093,0.000025
5,1.000000e-07,"[0.9999446566755096, 1.0001649240235981, 0.999...","[0.9999925955268383, 1.0000266217859428, 0.999...",0.000651,0.000057
6,1.000000e-08,"[0.9980691447937107, 1.0043984768201435, 1.000...","[0.9999918197260445, 1.0000302999882371, 0.999...",0.014829,0.000070
7,1.000000e-09,"[0.9958606357206831, 1.012916297544752, 0.9893...","[0.9999909989362111, 1.000035021328758, 0.9999...",0.030446,0.000075
8,1.000000e-10,"[0.9941692060537324, 1.0233416119405316, 0.997...","[0.9999917504471905, 1.0000304941385703, 0.999...",0.495167,0.000072
9,1.000000e-11,"[1.0304166128447558, 0.7724203955316078, 1.690...","[0.9999912559983974, 1.0000334479672655, 0.999...",1.963945,0.000073


In [63]:
fig = go.Figure()
fig.add_scatter(x=regularization_data['alpha'], y=regularization_data['error_1'], name='Метод №1')
fig.add_scatter(x=regularization_data['alpha'], y=regularization_data['error_2'], name='Метод №2')
fig.update_xaxes(type='log', tickformat='.0e', autorange='reversed', title='Параметр регуляризации')
fig.update_yaxes(title='Ошибка')
fig.update_layout(title='Зависимость ошибки от параметра регуляризации', template=None)
fig.show()

# fig.write_image('alpha-error.svg')

Unsupported

In [64]:
def find_min_alpha(size: int) -> pd.Series:
    regularization_data = get_regularization_data(size)

    min_error_1_arg = regularization_data['error_1'].argmin()
    min_error_2_arg = regularization_data['error_2'].argmin()

    min_error_1 = regularization_data.loc[min_error_1_arg, 'error_1']
    min_error_2 = regularization_data.loc[min_error_2_arg, 'error_2']

    alpha_1 = regularization_data.loc[min_error_1_arg, 'alpha']
    alpha_2 = regularization_data.loc[min_error_2_arg, 'alpha']

    return pd.Series([alpha_1, min_error_1, alpha_2, min_error_2])

In [65]:
regularization_min_data = pd.DataFrame(range(1, 11), columns=['n'])
regularization_min_data[['alpha_1', 'error_1', 'alpha_2', 'error_2']] = regularization_min_data.apply(
    func=lambda row: find_min_alpha(row.n),
    axis=1,
)
regularization_min_data

# regularization_min_data.to_csv('regularization_min_data.csv', index=False)

,n,alpha_1,error_1,alpha_2,error_2
0,1,1.000000e-11,1.000000e-11,1.000000e-11,1.000000e-11
1,2,1.000000e-11,6.184801e-11,1.000000e-11,1.137751e-11
2,3,1.000000e-11,3.047092e-10,1.000000e-11,1.082350e-11
3,4,1.000000e-11,9.884686e-10,1.000000e-11,1.020903e-11
4,5,1.000000e-11,7.051958e-07,1.000000e-11,6.285728e-11
5,6,1.000000e-07,1.026658e-05,1.000000e-11,1.430493e-09
6,7,1.000000e-07,1.800307e-05,1.000000e-09,2.385470e-08
7,8,1.000000e-07,2.030721e-05,1.000000e-06,1.072611e-06
8,9,1.000000e-06,4.116792e-05,1.000000e-07,6.983115e-07
9,10,1.000000e-06,9.305338e-05,1.000000e-05,1.712937e-05


In [66]:
fig = go.Figure()
fig.add_scatter(x=regularization_min_data['n'], y=regularization_min_data['error_1'], name='Метод №1')
fig.add_scatter(x=regularization_min_data['n'], y=regularization_min_data['error_2'], name='Метод №2')
fig.update_xaxes(title='Размер матрицы')
fig.update_yaxes(title='Минимальная ошибка', tickformat='.0e')
fig.update_layout(template=None, title='Зависимость минимальной ошибки от размера матрицы')
fig

# fig.write_image('n-min-error.svg')

Unsupported